In [1]:
import pandas as pd
from matplotlib import pyplot as plt
from plotnine import *

Matplotlib is building the font cache; this may take a moment.


ModuleNotFoundError: No module named 'plotnine'

In [13]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)

In [4]:
stock = 'AMZN'
balance_sheet = pd.read_csv(f'lots_of_csvs/{stock}_quarterly_balance-sheet.csv')
balance_sheet = balance_sheet.transpose().reset_index()
balance_sheet = balance_sheet.rename(columns=balance_sheet.iloc[0]).loc[1:]
balance_sheet = balance_sheet.rename(columns={'name':'Date'})
balance_sheet.replace(',','', regex=True, inplace=True)
for column in balance_sheet.columns:
    balance_sheet = balance_sheet.rename(columns={column:column.replace("\t", "")})
balance_sheet['ActualDate'] = pd.to_datetime(balance_sheet['Date'])
balance_sheet.drop('Date', axis=1, inplace=True)
balance_sheet['Date'] = balance_sheet['ActualDate'] + pd.Timedelta(61, unit='D')


cash_flow = pd.read_csv(f'lots_of_csvs/{stock}_quarterly_cash-flow.csv')
cash_flow = cash_flow.transpose().reset_index()
cash_flow = cash_flow.rename(columns=cash_flow.iloc[0]).loc[1:]
cash_flow = cash_flow.rename(columns={'name':'Date'})
cash_flow.replace(',','', regex=True, inplace=True)
cash_flow.drop(cash_flow.index[cash_flow['Date'] == 'ttm'], inplace=True)
for column in cash_flow.columns:
    cash_flow = cash_flow.rename(columns={column:column.replace("\t", "")})
cash_flow['ActualDate'] = pd.to_datetime(cash_flow['Date'])
cash_flow['Date'] = cash_flow['ActualDate'] + pd.Timedelta(61, unit='D')
cash_flow.drop('ActualDate', axis=1, inplace=True)


financials = pd.read_csv(f'lots_of_csvs/{stock}_quarterly_financials.csv')
financials = financials.transpose().reset_index()
financials = financials.rename(columns=financials.iloc[0]).loc[1:]
financials = financials.rename(columns={'name':'Date'})
financials.replace(',','', regex=True, inplace=True)
financials.drop(financials.index[financials['Date'] == 'ttm'], inplace=True)
for column in financials.columns:
    financials = financials.rename(columns={column:column.replace("\t", "")})
financials['ActualDate'] = pd.to_datetime(financials['Date'])
financials['Date'] = financials['ActualDate'] + pd.Timedelta(61, unit='D')
financials.drop('ActualDate', axis=1, inplace=True)

# valuation = pd.read_csv(f'lots_of_csvs/{stock}_quarterly_valuation_measures.csv')
# valuation = valuation.transpose().reset_index()
# valuation = valuation.rename(columns=valuation.iloc[0]).loc[1:]
# valuation = valuation.rename(columns={'name':'Date'})
# valuation.drop(valuation.index[valuation['Date'] == 'ttm'], inplace=True)
# valuation.replace(',','', regex=True, inplace=True)
# valuation['ActualDate'] = pd.to_datetime(valuation['Date'])
# valuation['Date'] = valuation['ActualDate'] + pd.Timedelta(61, unit='D')
# valuation.drop('ActualDate', axis=1, inplace=True)

price = pd.read_csv(f'lots_of_csvs/{stock}.csv')
price['Date'] = pd.to_datetime(price['Date'])
min = price['Date'].min()
max = price['Date'].max()
dates = pd.date_range(min, max, freq='D')
price.set_index('Date', inplace=True)
for i in range(0, len(dates)):
    try:
        price.loc[dates[i]]
    except:
        price.loc[dates[i]] = price.loc[dates[i-1]]

data = pd.merge(balance_sheet, cash_flow, on = "Date", how = "inner")
data = pd.merge(data, financials, on = "Date", how = "inner")
#data = pd.merge(data, valuation, on = "Date", how = "inner")
data = pd.merge(data, price, on = "Date", how = "inner")
data['Ticker'] = stock
data.set_index('Date', inplace=True)
data = data.sort_index(axis='index', ascending=True)

for column in data.columns:
    if (column == 'Date'):
        pass
    elif (column == 'Ticker'):
        data[column] = data[column].astype(str)
    else:
        data[column] = pd.to_numeric(data[column])

data = data.copy()
if ('ShareIssued' in data.columns or 'OrdinarySharesNumber' in data.columns):
    if 'OrdinarySharesNumber' not in data.columns:
        data['OrdinarySharesNumber'] = data['ShareIssued'] - data['TreasurySharesNumber']

    data['Price3Months'] = data['Adj Close'].shift(-1)
    data['Price6Months'] = data['Adj Close'].shift(-2)
    data['Price1Year'] = data['Adj Close'].shift(-4)
    data['Price3Years'] = data['Adj Close'].shift(-12)
    data['Price5Years'] = data['Adj Close'].shift(-20)
    data['Return3Months'] = data['Price3Months']/data['Adj Close']-1
    data['Return6Months'] = data['Price6Months']/data['Adj Close']-1
    data['Return1Year'] = data['Price1Year']/data['Adj Close']-1
    data['Return3Years'] = data['Price3Years']/data['Adj Close']-1
    data['Return5Years'] = data['Price5Years']/data['Adj Close']-1

    data['TTMEarnings'] = data['NetIncomeCommonStockholders'].rolling(window=4).sum()
    data['TTMSales'] = data['TotalRevenue'].rolling(window=4).sum()
    data['TTMEBITDA'] = data['NormalizedEBITDA'].rolling(window=4).sum()

    data['Revenue4QuartersAgo'] = data['TotalRevenue'].shift(4)
    data['RevenueLastYear'] = data['TTMSales'].shift(4)
    data['Earnings4QuartersAgo'] = data['NetIncomeCommonStockholders'].shift(4)
    data['EarningsLastYear'] = data['TTMEarnings'].shift(4)
    data['YoYRevenueGrowth'] = round((data['TotalRevenue']/data['Revenue4QuartersAgo']-1), 2)
    data['TTMRevenueGrowth'] = round((data['TTMSales']/data['RevenueLastYear']-1), 2)
    data['YoYEarningsGrowth'] = round((data['NetIncomeCommonStockholders']/data['Earnings4QuartersAgo']-1), 2)
    data['TTMEarningsGrowth'] = round((data['TTMEarnings']/data['EarningsLastYear']-1), 2)

    data['MarketCap'] = round((data['OrdinarySharesNumber'] * data['Close']), 0)
    data['EnterpriseValue'] = round((data['MarketCap'] - data['CashAndCashEquivalents'] + data['PreferredStock'] + data['TotalDebt']), 0)
    data['P/S'] = round((data['MarketCap'] / data['TTMSales']), 2)
    data['P/B'] = round((data['MarketCap'] / data['TangibleBookValue']), 2)
    data['P/E'] = round((data['MarketCap'] / data['TTMEarnings']), 2)
    data['EV/S'] = round((data['EnterpriseValue'] / data['TTMSales']), 2)
    data['EV/E'] = round((data['EnterpriseValue'] / data['TTMEarnings']), 2)
    data['EV/EBITDA'] = round((data['EnterpriseValue'] / data['TTMEBITDA']), 2)
else: 
    print(f'data missing for {stock}')



In [ ]:
display(data)

In [ ]:
stocks = pd.read_csv('Tickers.csv')

stocks = stocks['Stocks'].tolist()

df = pd.DataFrame()


for i in range(500, 1000):
    proceed = True
    stock = stocks[i]
    try:
        balance_sheet = pd.read_csv(f'lots_of_csvs/{stock}_quarterly_balance-sheet.csv')
        financials = pd.read_csv(f'lots_of_csvs/{stock}_quarterly_financials.csv')
        cash_flow = pd.read_csv(f'lots_of_csvs/{stock}_quarterly_cash-flow.csv')
        price = pd.read_csv(f'lots_of_csvs/{stock}.csv')

        balance_sheet = balance_sheet.transpose().reset_index()
        balance_sheet = balance_sheet.rename(columns=balance_sheet.iloc[0]).loc[1:]
        balance_sheet = balance_sheet.rename(columns={'name':'Date'})
        balance_sheet.replace(',','', regex=True, inplace=True)
        for column in balance_sheet.columns:
            balance_sheet = balance_sheet.rename(columns={column:column.replace("\t", "")})
        balance_sheet['ActualDate'] = pd.to_datetime(balance_sheet['Date'])
        balance_sheet.drop('Date', axis=1, inplace=True)
        balance_sheet['Date'] = balance_sheet['ActualDate'] + pd.Timedelta(61, unit='D')


        
        cash_flow = cash_flow.transpose().reset_index()
        cash_flow = cash_flow.rename(columns=cash_flow.iloc[0]).loc[1:]
        cash_flow = cash_flow.rename(columns={'name':'Date'})
        cash_flow.replace(',','', regex=True, inplace=True)
        cash_flow.drop(cash_flow.index[cash_flow['Date'] == 'ttm'], inplace=True)
        for column in cash_flow.columns:
            cash_flow = cash_flow.rename(columns={column:column.replace("\t", "")})
        cash_flow['ActualDate'] = pd.to_datetime(cash_flow['Date'])
        cash_flow['Date'] = cash_flow['ActualDate'] + pd.Timedelta(61, unit='D')
        cash_flow.drop('ActualDate', axis=1, inplace=True)


        
        financials = financials.transpose().reset_index()
        financials = financials.rename(columns=financials.iloc[0]).loc[1:]
        financials = financials.rename(columns={'name':'Date'})
        financials.replace(',','', regex=True, inplace=True)
        financials.drop(financials.index[financials['Date'] == 'ttm'], inplace=True)
        for column in financials.columns:
            financials = financials.rename(columns={column:column.replace("\t", "")})
        financials['ActualDate'] = pd.to_datetime(financials['Date'])
        financials['Date'] = financials['ActualDate'] + pd.Timedelta(61, unit='D')
        financials.drop('ActualDate', axis=1, inplace=True)

        # valuation = pd.read_csv(f'lots_of_csvs/{stock}_quarterly_valuation_measures.csv')
        # valuation = valuation.transpose().reset_index()
        # valuation = valuation.rename(columns=valuation.iloc[0]).loc[1:]
        # valuation = valuation.rename(columns={'name':'Date'})
        # valuation.drop(valuation.index[valuation['Date'] == 'ttm'], inplace=True)
        # valuation.replace(',','', regex=True, inplace=True)
        # valuation['ActualDate'] = pd.to_datetime(valuation['Date'])
        # valuation['Date'] = valuation['ActualDate'] + pd.Timedelta(61, unit='D')
        # valuation.drop('ActualDate', axis=1, inplace=True)

        
        price['Date'] = pd.to_datetime(price['Date'])
        min = price['Date'].min()
        max = price['Date'].max()
        dates = pd.date_range(min, max, freq='D')
        price.set_index('Date', inplace=True)
        for i in range(0, len(dates)):
            try:
                price.loc[dates[i]]
            except:
                price.loc[dates[i]] = price.loc[dates[i-1]]

        data = pd.merge(balance_sheet, cash_flow, on = "Date", how = "inner")
        data = pd.merge(data, financials, on = "Date", how = "inner")
        #data = pd.merge(data, valuation, on = "Date", how = "inner")
        data = pd.merge(data, price, on = "Date", how = "inner")
        data['Ticker'] = stock
        data.set_index('Date', inplace=True)
        data = data.sort_index(axis='index', ascending=True)

        

        for column in data.columns:
            if (column == 'Date'):
                pass
            elif (column == 'Ticker'):
                data[column] = data[column].astype(str)
            else:
                data[column] = pd.to_numeric(data[column])

        data = data.copy()
    except:
        print(f'Missing a file for {stock}')
        proceed = False

    if (proceed == True):
        
        try:
            if ('ShareIssued' in data.columns or 'OrdinarySharesNumber' in data.columns):
                if 'OrdinarySharesNumber' not in data.columns:
                    data['OrdinarySharesNumber'] = data['ShareIssued'] - data['TreasurySharesNumber']

                data['Price3Months'] = data['Adj Close'].shift(-1)
                data['Price6Months'] = data['Adj Close'].shift(-2)
                data['Price1Year'] = data['Adj Close'].shift(-4)
                data['Price3Years'] = data['Adj Close'].shift(-12)
                data['Price5Years'] = data['Adj Close'].shift(-20)
                data['Return3Months'] = data['Price3Months']/data['Adj Close']-1
                data['Return6Months'] = data['Price6Months']/data['Adj Close']-1
                data['Return1Year'] = data['Price1Year']/data['Adj Close']-1
                data['Return3Years'] = data['Price3Years']/data['Adj Close']-1
                data['Return5Years'] = data['Price5Years']/data['Adj Close']-1

                data['TTMEarnings'] = data['NetIncomeCommonStockholders'].rolling(window=4).sum()
                data['TTMSales'] = data['TotalRevenue'].rolling(window=4).sum()
                #data['TTMEBITDA'] = data['NormalizedEBITDA'].rolling(window=4).sum()

                data['Revenue4QuartersAgo'] = data['TotalRevenue'].shift(4)
                data['RevenueLastYear'] = data['TTMSales'].shift(4)
                data['Earnings4QuartersAgo'] = data['NetIncomeCommonStockholders'].shift(4)
                data['EarningsLastYear'] = data['TTMEarnings'].shift(4)
                data['YoYRevenueGrowth'] = round((data['TotalRevenue']/data['Revenue4QuartersAgo']-1), 2)
                data['TTMRevenueGrowth'] = round((data['TTMSales']/data['RevenueLastYear']-1), 2)
                data['YoYEarningsGrowth'] = round((data['NetIncomeCommonStockholders']/data['Earnings4QuartersAgo']-1), 2)
                data['TTMEarningsGrowth'] = round((data['TTMEarnings']/data['EarningsLastYear']-1), 2)

                data['MarketCap'] = round((data['OrdinarySharesNumber'] * data['Close']), 0)
                #data['EnterpriseValue'] = round((data['MarketCap'] - data['CashAndCashEquivalents'] + data['PreferredStock'] + data['TotalDebt']), 0)
                data['P/S'] = round((data['MarketCap'] / data['TTMSales']), 2)
                data['P/B'] = round((data['MarketCap'] / data['TangibleBookValue']), 2)
                data['P/E'] = round((data['MarketCap'] / data['TTMEarnings']), 2)
                #data['EV/S'] = round((data['EnterpriseValue'] / data['TTMSales']), 2)
                #data['EV/E'] = round((data['EnterpriseValue'] / data['TTMEarnings']), 2)
                #data['EV/EBITDA'] = round((data['EnterpriseValue'] / data['TTMEBITDA']), 2)

                df = pd.concat([df, data], axis=0)
                print(stock)
            else: 
                print(f'data missing for {stock}')
        except:
            print('problems during feature engineering')

df.to_parquet('all_stocks_v2.parquet')


    

In [6]:
stocks = pd.read_csv('Tickers.csv')

stocks = stocks['Stocks'].tolist()

In [ ]:
display(balance_sheet)
display(cash_flow)
display(financials)
#display(valuation)
display(price)

In [149]:
data = pd.merge(balance_sheet, cash_flow, on = "Date", how = "inner")
data = pd.merge(data, financials, on = "Date", how = "inner")
#data = pd.merge(data, valuation, on = "Date", how = "inner")
data = pd.merge(data, price, on = "Date", how = "inner")
data['Ticker'] = stock
data.set_index('Date', inplace=True)
data = data.sort_index(axis='index', ascending=True)


In [9]:
def move_columns(list_of_columns, df):
    for column in list_of_columns:
        just_a_column = df[column]
        df = df.drop(columns=[column])
        df.insert(loc=0, column=column, value=just_a_column)
    return df

In [105]:
data = move_columns(['MarketCap', 'Close', 'OrdinarySharesNumber','Date'], data)

In [ ]:
for column in df.columns:
    print(column)

In [15]:
df.to_parquet('all_stocks.parquet')

In [8]:
test_df = pd.read_parquet('all_stocks_v2.parquet')

In [ ]:


df = test_df.dropna(subset=['TotalAssets', 'OperatingCashFlow', 'TotalRevenue'])

display(df)



In [15]:
organized_df = move_columns(['OrdinarySharesNumber','ActualDate', 'Adj Close','Volume','Ticker','Price3Months','Price6Months','Price1Year','Price3Years','Price5Years','TTMEarnings','TTMSales','Revenue4QuartersAgo','RevenueLastYear','Earnings4QuartersAgo','EarningsLastYear','YoYRevenueGrowth','TTMRevenueGrowth','YoYEarningsGrowth','TTMEarningsGrowth','MarketCap','Return3Months','Return6Months','Return1Year','Return3Years','Return5Years','P/S','P/B','P/E','Ticker'], data)

In [16]:
display(organized_df)

,Ticker,P/E,P/B,P/S,Return5Years,Return3Years,Return1Year,Return6Months,Return3Months,MarketCap,TTMEarningsGrowth,YoYEarningsGrowth,TTMRevenueGrowth,YoYRevenueGrowth,EarningsLastYear,Earnings4QuartersAgo,RevenueLastYear,Revenue4QuartersAgo,TTMSales,TTMEarnings,Price5Years,Price3Years,Price1Year,Price6Months,Price3Months,Volume,Adj Close,ActualDate,OrdinarySharesNumber,TotalAssets,CurrentAssets,CashCashEquivalentsAndShortTermInvestments,CashAndCashEquivalents,OtherShortTermInvestments,Receivables,AccountsReceivable,GrossAccountsReceivable,AllowanceForDoubtfulAccountsReceivable,Inventory,FinishedGoods,PrepaidAssets,CurrentDeferredAssets,CurrentDeferredTaxesAssets,OtherCurrentAssets,TotalNonCurrentAssets,NetPPE,GrossPPE,Properties,LandAndImprovements,BuildingsAndImprovements,MachineryFurnitureEquipment,OtherProperties,ConstructionInProgress,AccumulatedDepreciation,GoodwillAndOtherIntangibleAssets,Goodwill,OtherIntangibleAssets,NonCurrentDeferredAssets,NonCurrentDeferredTaxesAssets,OtherNonCurrentAssets,TotalLiabilitiesNetMinorityInterest,CurrentLiabilities,PayablesAndAccruedExpenses,Payables,AccountsPayable,CurrentAccruedExpenses,CurrentDebtAndCapitalLeaseObligation,CurrentDebt,CurrentDeferredLiabilities,CurrentDeferredRevenue,OtherCurrentLiabilities,TotalNonCurrentLiabilitiesNetMinorityInterest,LongTermDebtAndCapitalLeaseObligation,LongTermDebt,LongTermCapitalLeaseObligation,NonCurrentDeferredLiabilities,NonCurrentDeferredTaxesLiabilities,TradeandOtherPayablesNonCurrent,OtherNonCurrentLiabilities,TotalEquityGrossMinorityInterest,StockholdersEquity,CapitalStock,PreferredStock,CommonStock,AdditionalPaidInCapital,RetainedEarnings,TreasuryStock,GainsLossesNotAffectingRetainedEarnings,OtherEquityInterest,TotalCapitalization,CommonStockEquity,CapitalLeaseObligations,NetTangibleAssets,WorkingCapital,InvestedCapital,TangibleBookValue,TotalDebt,NetDebt,ShareIssued,TreasurySharesNumber,OperatingCashFlow,CashFlowFromContinuingOperatingActivities,NetIncomeFromContinuingOperations,OperatingGainsLosses,GainLossOnInvestmentSecurities,DepreciationAmortizationDepletion,DepreciationAndAmortization,Depreciation,AmortizationCashFlow,AmortizationOfIntangibles,DeferredTax,DeferredIncomeTax,StockBasedCompensation,ExcessTaxBenefitFromStockBasedCompensation,OtherNonCashItems,ChangeInWorkingCapital,ChangeInReceivables,ChangesInAccountReceivables,ChangeInInventory,ChangeInPrepaidAssets,ChangeInPayablesAndAccruedExpense,ChangeInPayable,ChangeInAccountPayable,ChangeInAccruedExpense,ChangeInOtherCurrentAssets,ChangeInOtherCurrentLiabilities,ChangeInOtherWorkingCapital,InvestingCashFlow,CashFlowFromContinuingInvestingActivities,NetPPEPurchaseAndSale,PurchaseOfPPE,SaleOfPPE,NetBusinessPurchaseAndSale,PurchaseOfBusiness,NetInvestmentPurchaseAndSale,PurchaseOfInvestment,SaleOfInvestment,NetOtherInvestingChanges,FinancingCashFlow,CashFlowFromContinuingFinancingActivities,NetIssuancePaymentsOfDebt,NetLongTermDebtIssuance,LongTermDebtIssuance,LongTermDebtPayments,NetShortTermDebtIssuance,ShortTermDebtIssuance,ShortTermDebtPayments,NetCommonStockIssuance,CommonStockIssuance,CommonStockPayments,ProceedsFromStockOptionExercised,NetOtherFinancingCharges,EndCashPosition,ChangesInCash,EffectOfExchangeRateChanges,BeginningCashPosition,OtherCashAdjustmentOutsideChangeinCash,IncomeTaxPaidSupplementalData,InterestPaidSupplementalData,CapitalExpenditure,IssuanceOfCapitalStock,IssuanceOfDebt,RepaymentOfDebt,RepurchaseOfCapitalStock,FreeCashFlow,TotalRevenue,OperatingRevenue,CostOfRevenue,GrossProfit,OperatingExpense,SellingGeneralAndAdministration,GeneralAndAdministrativeExpense,OtherGandA,SellingAndMarketingExpense,ResearchAndDevelopment,DepreciationAmortizationDepletionIncomeStatement,DepreciationAndAmortizationInIncomeStatement,Amortization,AmortizationOfIntangiblesIncomeStatement,OtherOperatingExpenses,OperatingIncome,NetNonOperatingInterestIncomeExpense,InterestIncomeNonOperating,InterestExpenseNonOperating,OtherIncomeExpense,GainOnSaleOfSecurity,EarningsFromEquityInteres